In [2]:
!pip install -q "transformers>=4.44.0" "datasets>=2.20.0" "accelerate>=0.33.0" \
              "huggingface_hub" "evaluate" --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 14.7 MB/s eta 0:00:00


In [3]:
!pip install -q trl bitsandbytes
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format
import torch
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 16.0 MB/s eta 0:00:00


In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

device


'cuda'

In [5]:
dataset_id = "prithivMLmods/Deepthink-Reasoning"

ds = load_dataset(dataset_id)
ds


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

dataset/0000.parquet:   0%|          | 0.00/401k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/251 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response'],
        num_rows: 251
    })
})

In [6]:
print("Splits:", ds.keys())
print("Columns:", ds["train"].column_names)

ds["train"][0]


Splits: dict_keys(['train'])
Columns: ['prompt', 'response']


{'prompt': 'Search Algorithms',
 'response': '<|thinking|>\n## Analyzing the request\nThe user is asking about search algorithms, which are methods for finding specific elements within a data structure (e.g., an array or list).\n\n## Planning the solution\nI should:\n\n1. Provide an overview of common search algorithms.\n2. Explain how each algorithm works, including its time and space complexity.\n3. Implement examples of key search algorithms in Python.\n\n## Deciding on the approach\nI will cover the following search algorithms:\n\n1. **Linear Search**: Simple but inefficient for large datasets.\n2. **Binary Search**: Efficient for sorted datasets.\n3. **Jump Search**: A compromise between linear and binary search.\n4. **Exponential Search**: Useful for unbounded or large datasets.\n5. **Interpolation Search**: Effective for uniformly distributed data.\n6. **Fibonacci Search**: A variant of binary search using Fibonacci numbers.\n\nI will provide Python implementations for each algo

In [10]:
MAX_LEN = 512

def tokenize_function(examples):
    prompts = [p.strip() for p in examples["prompt"]]
    responses = [r.strip() for r in examples["response"]]
    texts = [
        tokenizer.apply_chat_template(
            [
                {"role": "user", "content": p},
                {"role": "assistant", "content": r},
            ],
            tokenize=False
        )
        for p, r in zip(prompts, responses)
    ]

    tokenized = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN,
    )
    return tokenized

ds_tokenized = ds.map(tokenize_function, batched=True)
ds_tokenized


Map:   0%|          | 0/251 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response', 'input_ids', 'attention_mask'],
        num_rows: 251
    })
})

In [13]:
from transformers import TrainingArguments
import torch

use_bf16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()

training_args = TrainingArguments(
    output_dir="outputs-smollm2-deepthink",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=60,             # total steps (small for demo – you can increase)
    learning_rate=2e-4,
    fp16=not use_bf16,
    bf16=use_bf16,
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    report_to="none",         # change to "wandb" if you use Weights & Biases
)


In [15]:
model_name = "HuggingFaceTB/SmolLM2-360M"

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    device_map="auto",
    torch_dtype="auto"
)
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=model_name
)

model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

trainer = SFTTrainer(
    model=model,
    train_dataset=ds_tokenized["train"],
    args=training_args,
)



/usr/local/lib/python3.12/dist-packages/trl/models/utils.py:117: FutureWarning: The `setup_chat_format` function is deprecated and will be removed in version 0.26.0. Please use `clone_chat_template` instead.
  warnings.warn(


Truncating train dataset:   0%|          | 0/251 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [16]:
use_bf16 = torch.cuda.is_bf16_supported()

# Choose optimizer based on device availability
optimizer_type = "adamw_8bit" if torch.cuda.is_available() else "adamw_torch"

training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=60,
    learning_rate=2e-4,
    fp16=not use_bf16,
    bf16=use_bf16,
    logging_steps=1,
    optim=optimizer_type,
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs-smollm2-deepthink",
    report_to="none",
)

In [17]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=ds_tokenized["train"],
    args=training_args,
)

trainer


The model is already on multiple devices. Skipping the move to device specified in `args`.


In [18]:
trainer.train()


Step,Training Loss
1,1.838100
2,1.661200
3,1.514200
4,1.369900
5,1.412800
6,1.178600
7,1.311200
8,1.079200
9,1.225600
10,0.996700


TrainOutput(global_step=60, training_loss=0.8921435803174973, metrics={'train_runtime': 267.5087, 'train_samples_per_second': 1.794, 'train_steps_per_second': 0.224, 'total_flos': 459115683840000.0, 'train_loss': 0.8921435803174973, 'epoch': 1.8888888888888888})

In [19]:
save_directory = "/content/smollm2-deepthink-ft"

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

save_directory


'/content/smollm2-deepthink-ft'

In [22]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

ft_model = AutoModelForCausalLM.from_pretrained(
    save_directory,
    torch_dtype="auto",
    device_map="auto"
)
ft_tokenizer = AutoTokenizer.from_pretrained(save_directory)

reasoning_pipe = pipeline(
    "text-generation",
    model=ft_model,
    tokenizer=ft_tokenizer,

)

test_question = "Write a step-by-step reasoning to solve: 24 is divided by 3 and then multiplied by 5. What is the result?"

out = reasoning_pipe(test_question, max_new_tokens=200, do_sample=True, temperature=0.7)
print(out[0]["generated_text"])


Device set to use cuda:0


Write a step-by-step reasoning to solve: 24 is divided by 3 and then multiplied by 5. What is the result? Write only the final result with a complete explanation of the steps used.

### Example:

To solve the problem, we need to:

1. Divide 24 by 3.
2. Multiply the result by 5.

### Solution:

1. Divide 24 by 3. The result is 8 (24 divided by 3).
2. Multiply 8 by 5. The result is 40.

## Step-by-Step Reasoning:

### Explanation of the Steps:

1. **Divide by 3**: We divide 24 by 3, which results in 8, giving us our final result of 8.
2. **Multiply by 5**: We multiply the result of step 1 by 5, which gives us the final result of 40.

## Example:

Solve: 24 ÷
